In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../')

In [3]:
import torch
from experiments import intervention_utils
import os
from cem.data.CUB200.cub_loader import load_data, find_class_imbalance
from torchvision.models import resnet50, resnet34
from cem.models.cem import ConceptEmbeddingModel
import pytorch_lightning as pl
import numpy as np
import cem.train.training as cem_train
import re
import random
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from cem.models.vae_model import *
from cem.metrics.homogeneity import embedding_homogeneity

2023-04-22 18:10:43.397719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 18:10:51.593707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ../../anaconda3/lib
2023-04-22 18:10:51.598270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: ../../anaconda3/lib
2023-04-22 18:10:51.598288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38

In [4]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)

In [5]:
CUB_DIR = 'CUB/'
BASE_DIR = os.path.join(CUB_DIR, 'images/CUB_200_2011/images')
num_workers=8
n_tasks = 200
n_concepts = 112
gpu = 0
sample_train = 0.1
concept_group_map = intervention_utils.CUB_CONCEPT_GROUP_MAP
num_epochs = 100
seed = 42

In [3]:
import experiments.extract_cem_concepts

In [5]:
train, val, test = experiments.extract_cem_concepts.generate_data_loaders_dsprites("")

In [24]:
import pickle
a = pickle.load(open("dsprites/preprocessed/test.pkl","rb"))

In [26]:
a[0]

{'id': 715240,
 'img_path': 'dsprites/images/715240.png',
 'class_label': 147786,
 'attribute_label': [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0]}

In [10]:
next(iter(val))

[tensor([[[[-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           ...,
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500]],
 
          [[-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           ...,
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [ 0.2500,  0.2500,  0.2500,  ..., -0.2500, -0.2500, -0.2500]],
 
          [[-0.2500, -0.2500, -0.2500,  ..., -0.2500, -0.2500, -0.2500],
           [-

In [6]:
train_data_path = os.path.join(CUB_DIR, 'preprocessed/train.pkl')
val_data_path = train_data_path.replace('train.pkl', 'val.pkl')
test_data_path = train_data_path.replace('train.pkl', 'test.pkl')

In [7]:
config = joblib.load("results/ConceptEmbeddingModelNew_resnet34_fold_1_experiment_config.joblib")

In [8]:
if config['weight_loss']:
    imbalance = find_class_imbalance(train_data_path, True)
else:
    imbalance = None

In [9]:
selected_concepts = np.arange(n_concepts)
def subsample_transform(sample):
    if isinstance(sample, list):
        sample = np.array(sample)
    return sample[selected_concepts]

In [10]:
train_dl = load_data(
    pkl_paths=[train_data_path],
    use_attr=True,
    no_img=False,
    batch_size=config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//",""),
)
val_dl = load_data(
    pkl_paths=[val_data_path],
    use_attr=True,
    no_img=False,
    batch_size=config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//",""))
test_dl = load_data(
    pkl_paths=[test_data_path],
    use_attr=True,
    no_img=False,
    batch_size=config['batch_size'],
    uncertain_label=False,
    n_class_attr=2,
    image_dir='images',
    resampling=False,
    root_dir='.',
    num_workers=config['num_workers'],
    concept_transform=subsample_transform,
    path_transform=lambda path: path.replace("CUB//","")
)

In [11]:
model = intervention_utils.load_trained_model(
        config=config,
        n_tasks=n_tasks,
        n_concepts=n_concepts,
        result_dir="results/",
        split=0,
        imbalance=imbalance,
        intervention_idxs=[],
        train_dl=train_dl,
        sequential=False,
        independent=False,
    )

/rds/user/njr61/hpc-work/anaconda3/envs/cem/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/rds/user/njr61/hpc-work/anaconda3/envs/cem/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
x_all = []
y_all = []
c_all = []

In [47]:
for batch in val_dl:
    x, y, c = batch
    x_all.append(x)
    y_all.append(y)
    c_all.append(c)

In [48]:
x_all = torch.cat(x_all, dim=0)
y_all = torch.cat(y_all, dim=0)
c_all = torch.cat(c_all, dim=0)

In [49]:
results = model(x_all)

In [50]:
[results[i].shape for i in range(len(results))]

[torch.Size([1198, 112]),
 torch.Size([1198, 224]),
 torch.Size([1198, 200]),
 torch.Size([112, 1198, 4])]

In [51]:
c_pred, c_emb, y_pred, _ = results

In [52]:
c_pred = c_pred.detach()
c_emb = c_emb.detach()

In [56]:
output

(0.18452380952381, 0.018697539184584857)

In [15]:
model.concept_folder_location = "cem_concepts/cub/42"

In [16]:
model.experiment_name = "cub"

In [17]:
trainer = pl.Trainer(
            gpus=gpu,
        )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [18]:
trainer.test(model, val_dl, verbose=False,)

Testing: 0it [00:00, ?it/s]

[{'test_c_accuracy': 0.95804762840271,
  'test_c_auc': 0.9330776333808899,
  'test_c_f1': 0.9353863000869751,
  'test_y_accuracy': 0.7562604546546936,
  'test_y_auc': 0.0,
  'test_y_f1': 0.6388682126998901,
  'test_concept_loss': 0.5386481881141663,
  'test_task_loss': 1.2954771518707275,
  'test_loss': 3.988718271255493,
  'test_avg_c_y_acc': 0.8571540713310242}]

In [19]:
model.write_concepts()

## Create test npy files

In [4]:
val_file_loc = "../../spurious-concepts/ConceptBottleneck/valid_c.npy"

In [6]:
valid_c_predicted = np.load(open(val_file_loc,"rb"))

In [8]:
import pickle 
valid_pkl = pickle.load(open("CUB/preprocessed/val.pkl","rb"))

In [14]:
valid_c = np.stack([i['attribute_label'] for i in valid_pkl])

In [16]:
np.mean((valid_c_predicted-valid_c)**2)

0.015027710813242504

In [17]:
test_file_loc = "../../spurious-concepts/ConceptBottleneck/test_c.npy"

In [18]:
test_c_predicted = np.load(open(test_file_loc,"rb"))

In [19]:
test_pkl = pickle.load(open("CUB/preprocessed/test.pkl","rb"))

In [20]:
test_c = np.stack([i['attribute_label'] for i in test_pkl])

In [21]:
np.mean((test_c_predicted-test_c)**2)

0.034268729168837234

In [22]:
train_file_loc = "../../spurious-concepts/ConceptBottleneck/train_c.npy"

In [23]:
train_c_predicted = np.load(open(train_file_loc,"rb"))

In [24]:
train_pkl = pickle.load(open("CUB/preprocessed/train.pkl","rb"))

In [25]:
train_c = np.stack([i['attribute_label'] for i in train_pkl])

In [26]:
np.mean((train_c_predicted-train_c)**2)

0.014827635405120727

In [28]:
from collections import Counter

In [29]:
Counter([i['class_label'] for i in train_pkl])

Counter({21: 23,
         51: 23,
         62: 27,
         168: 22,
         0: 26,
         70: 22,
         24: 27,
         100: 22,
         96: 28,
         126: 26,
         186: 27,
         188: 20,
         34: 21,
         160: 22,
         148: 25,
         48: 22,
         53: 23,
         122: 24,
         174: 22,
         31: 19,
         32: 28,
         132: 24,
         153: 26,
         12: 27,
         68: 22,
         15: 24,
         102: 26,
         144: 23,
         80: 26,
         30: 22,
         115: 26,
         123: 25,
         177: 26,
         134: 26,
         166: 24,
         103: 24,
         184: 25,
         193: 23,
         147: 21,
         1: 22,
         187: 23,
         79: 28,
         107: 26,
         178: 25,
         47: 22,
         55: 27,
         141: 20,
         119: 25,
         142: 25,
         39: 24,
         88: 23,
         196: 24,
         151: 24,
         63: 26,
         170: 27,
         179: 29,
         57: 23,
 

In [30]:
Counter([i['class_label'] for i in test_pkl])

Counter({0: 30,
         1: 30,
         2: 28,
         3: 30,
         4: 14,
         5: 11,
         6: 23,
         7: 18,
         8: 29,
         9: 30,
         10: 30,
         11: 26,
         12: 30,
         13: 30,
         14: 28,
         15: 28,
         16: 27,
         17: 15,
         18: 29,
         19: 29,
         20: 30,
         21: 26,
         22: 29,
         23: 22,
         24: 30,
         25: 30,
         26: 30,
         27: 29,
         28: 30,
         29: 30,
         30: 30,
         31: 23,
         32: 29,
         33: 29,
         34: 30,
         35: 30,
         36: 29,
         37: 30,
         38: 29,
         39: 30,
         40: 30,
         41: 30,
         42: 29,
         43: 30,
         44: 30,
         45: 30,
         46: 30,
         47: 30,
         48: 30,
         49: 30,
         50: 30,
         51: 30,
         52: 30,
         53: 30,
         54: 30,
         55: 30,
         56: 30,
         57: 28,
         58: 30,
       